In [16]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

def makeKeyPair():
    private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    )

    public_key = private_key.public_key()

    return private_key, public_key

def signSomething(private_key, something):
    signature = private_key.sign(
        something,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    
    return signature

def verifySomething(public_key, signature, something):
    try:
        public_key.verify(
            signature,
            something,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
    except:
        print("Invalid Signature")


In [20]:
# making the key pairs
privkey, pubkey = makeKeyPair()
privkey2, pubkey2 = makeKeyPair()

# signing a message with the private key
message = b"A message I want to sign"
signature = signSomething(privkey2, message)

# verifying it
verifySomething(pubkey2, signature, message)